# Configurando e testando prompts

In [1]:
#@title Instalar dependências e carregar pacotes do git
# carregando o get_git para importar utilitários do github
import os
if not os.path.isfile('./get_git.py'):
   !curl https://raw.githubusercontent.com/luizanisio/llms/refs/heads/main/util/get_git.py -o ./get_git.py
import get_git
get_git.deps() # instala unsloth, Transformers, Rouge e Levenshtein se precisar
Util = get_git.sync_git_util()
Util.dados_python()
Prompt = get_git.sync_git_prompt()
UtilAgentes = get_git.sync_git_agentes()

✅ Instalação já realizada nesse ambiente _o/
[OK] Dependências de análise instaladas _o/
✅ Instalação já realizada nesse ambiente _o/
[OK] Dependências de treinamento instaladas _o/
[OK] Util carregado de /content/src (cache)
Util carregado corretamente em 2025-08-03 17:51:46!
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
[OK] Prompt carregado de /content/src (cache)
Transformers: 4.53.3 2025.7.1
Tourch: 2.7.0+cu126
[OK] UtilAgentes carregado de /content/src (git)
UtilAgentes carregado corretamente em 2025-08-03 17:52:11!


In [2]:
#@title Escolhe e carrega o modelo
pg3 = Prompt(modelo=4, max_seq_length=4096)

PromptGemma3: carregando modelo google/gemma-3-4b-it ..
==((====))==  Unsloth 2025.7.1: Fast Gemma3 patching. Transformers: 4.53.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [3]:
#@title Pergunta ao modelo
res = pg3.prompt('Complete os próximos dois números da sequência e explique de forma objetiva em uma frase: 1, 1, 2, 3, 5, 8 ...')
print('RESPOSTA:', res)

RESPOSTA: Os próximos dois números da sequência são 13 e 21.

Em uma frase objetiva: A sequência representa a sequência de Fibonacci, onde cada número é a soma dos dois números anteriores.


In [4]:
#@title Resposta em json do modelo
import json
pergunta = '''Retorne um json válido com a estrutrua {"mensagem": com a mensagem do usuário, "itens": com uma lista de itens quando ele enumerar algo }
              Mensagem do usuário: Eu preciso comprar abacaxi, pera e 2L de leite.'''
r = pg3.prompt_to_json(pergunta)
print(json.dumps(r, indent=2, ensure_ascii=False))

{
  "mensagem": "Eu preciso comprar abacaxi, pera e 2L de leite.",
  "itens": [
    "abacaxi",
    "pera",
    "2L de leite"
  ],
  "usage": {
    "input_tokens": 64,
    "output_tokens": 50,
    "time": 4.438987970352173
  }
}


# Configurando agentes

In [6]:
#@title Tarefa básica de exemplo
servicos = UtilAgentes.AgentesToolsBasicos.get_servicos_basicos()

tarefa = UtilAgentes.Tarefa(nome='Tarefa 001',
                descricao='Descobrir que dia é hoje e encontrar informações sobre o que é Xibunfa e como podemos usar Xibunfa para limpar o chão.',
                objetivo='Descrever o que é Xibunfa e explicar como Xibunfa pode ser usada na limpeza do chão.',
                servicos_disponiveis=servicos,
                call_llm = pg3.prompt_to_json)

rt = UtilAgentes.ResolverTarefas([tarefa])
rt.resolver()

17:55:53|RESOLVENDO TAREFA - : Tarefa 001 com 2 serviços disponíveis
17:55:53|DESCOBERTA - : Descobrindo serviços para a tarefa Tarefa 001
17:56:05|DESCOBERTA CONCLUÍDA - : Serviços descobertos para a tarefa Tarefa 001: 2
17:56:05|SERVIÇO SUGERIDO - : Busca - Para descobrir o que é Xibunfa e como ele pode ser usado na limpeza do chão.
17:56:05|SERVIÇO SUGERIDO - : DataHora - Para determinar o dia atual, necessário para responder à primeira parte da tarefa.
17:56:05|ACIONANDO SERVIÇOS - : Busca,DataHora
17:56:05|PREPARANDO SERVIÇO - : Busca: {"palavras": ["Xibunfa", "limpeza do chão"]}
17:56:05|SERVIÇO ACIONADO - : Busca: {"palavras": ["Xibunfa", "limpeza do chão"]}
17:56:05|PREPARANDO SERVIÇO - : DataHora: {}
17:56:05|SERVIÇO ACIONADO - : DataHora: {}
17:56:05|CONHECIMENTO - : Calendário com dia, mês e ano | pg. None | score 0
ACIONANDO AGENTE DE TAREFA: Padrão
AVALIANDO RESPOSTA LLM:
{
  "solucao_tarefa": "Hoje é 03 de Agosto de 2025. De acordo com a Base de Conhecimento, Xibunfa é um

In [7]:
#@title Imprimindo solução da tarefa
tarefa.print(True)

TAREFA: Tarefa 001
DESCRIÇÃO: Descobrir que dia é hoje e encontrar informações sobre o que é Xibunfa e como podemos usar Xibunfa para limpar o chão.
OBJETIVO: Descrever o que é Xibunfa e explicar como Xibunfa pode ser usada na limpeza do chão.
---------------------------------------
CONHECIMENTO UTILIZADO:
	 - Calendário com dia, mês e ano (pg: None | sc: 0): Hoje é o dia 03 de August de 2025
---------------------------------------
SOLUÇÃO COMPLETA: Hoje é 03 de Agosto de 2025. De acordo com a Base de Conhecimento, Xibunfa é um produto que pode ser usado para limpar o chão.
